# Make Prediction

This notebook allows you to make predictions with a trained neural network for a dataset created with prepare_noise_set.ipynb

In [ ]:
import os
os.chdir('../')

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
#import utils
import argparse
import torch.utils.data as data_utils
from torch import nn
from torch import optim
from data_loader.data_loader import create_loader
import trainer
import logger
import copy
import json
import pandas as pd
from sigpyproc.Readers import FilReader as reader
plt.rcParams["figure.figsize"] = (20,10)

In [3]:
cuda = 1
np.random.seed(2)
torch.manual_seed(1)  # reproducible
if cuda:
    torch.cuda.manual_seed(1)  # reproducible
device = torch.device("cuda:0" if cuda else "cpu")

In [4]:
full_vals = []

In [22]:
model = './trained_models/test_model_2.pt'
        
model_name = model.split('/')[-1].split('.pt')[0]
csv = 'noiseset_noise_sample.csv'
csv_name = csv.split('.')[0]
args = argparse.Namespace()
args.samples = 0
args.path = csv
args.noise = [0,0,0,0,0]
args.batch = 1
args.p = False
args.bandpass = False

In [6]:
net = torch.load(model)
net.mode = 'regressor'
net.epoch = 0
for param in net.parameters():
    param.requires_grad = False
train_loader_2, valid_loader_2, minimal_goal, mean_period, mean_dm, _,_,_  = create_loader(csv, None, args.samples, net.input_shape[1], args.batch, mean_period=net.mean_vals[0],
                                                                                mean_dm=net.mean_vals[1], val_frac=1, edge=net.edge
                                                                                   ,test=True, test_samples=1, shuffle_valid=False)
logging = logger.logger(args.p, 'yoo')
test_loader=None
train_net = trainer.trainer(net, train_loader_2, valid_loader_2, valid_loader_2,logging,
                            device, args.bandpass, 0, args.noise, 10, 0, reduce_test=False)
print(net.multi_class.parameter)
loss = train_net.run('test', 1, only_class=1, print_progress=False)
print(loss)
target = np.asarray(logging.target_stack[2:])
output = np.asarray(logging.out_stack[2:])
target_class = target[:,2:]
output_class = output[:,:2]
output_single = np.asarray(logging.out_single_stack)[:,:,:2]
output_single_period = np.asarray(logging.out_single_stack)[:,:,2]
output_single_soft = F.softmax(torch.Tensor(output_single), dim=2)
max_single, _ = output_single_soft[:,:,1].max(dim=1)
softmax = nn.Softmax(dim=1)
output_class = softmax(torch.Tensor(output_class))
#print(output_class.shape, output_single.shape)
#print(target_class.shape, output_class[:,1:].shape, output_single_soft[:,0,1].shape)
concat = np.concatenate((target_class, output_class[:,1:], max_single[:,None], output_single_soft[:,:,1], output_single_period), axis=1)
concat_sorted = concat[concat[:,4].argsort()][::-1]

Number of samples: 40
Using existing mean period and dm: 0.6393900000000001 307.75100000000003
noiseset_noise_sample.csv
[0, 0]
DM Ranges:
None
Parameter containing:
tensor([0.9030, 1.1658], device='cuda:0')
tensor(1.4026, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6045, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4280, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5830, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3622, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9160, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5625, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3118, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2533, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3337, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2929, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3260, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3448, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2544, device='cu

In [13]:
df = pd.read_csv('./datasets/'+csv)

In [18]:
prediction_array = output_class.numpy()[:,1]

In [23]:
df['PulsarPrediction'] = prediction_array

In [24]:
!mkdir predictions
df.to_csv(f"./predictions/{csv_name}_{model_name}.csv")

mkdir: cannot create directory ‘predictions’: File exists


In [36]:
pd.set_option('display.max_colwidth', None)
df[df['PulsarPrediction']>0.5][['FileName', 'PulsarPrediction']]

,FileName,PulsarPrediction
0,/data/lkuenkel/data/pipeline_test/noise_data/PM0020_04621.fil,0.754045
5,/data/lkuenkel/data/pipeline_test/noise_data/PM0005_00171.fil,0.599898
20,/data/lkuenkel/data/pipeline_test/noise_data/PM0012_00791.fil,0.597852
24,/data/lkuenkel/data/pipeline_test/noise_data/PM0012_03091.fil,0.869752
31,/data/lkuenkel/data/pipeline_test/noise_data/PM0013_02151.fil,0.636466
33,/data/lkuenkel/data/pipeline_test/noise_data/PM0016_02231.fil,0.579139
38,/data/lkuenkel/data/pipeline_test/noise_data/PM0014_03071.fil,0.615183
